# Twitter NLP

In [2]:
import pandas as pd
import numpy as np
import pickle
import time
from itertools import chain
from tqdm import tqdm_notebook
from ast import literal_eval

from tqdm import tqdm

tqdm.pandas()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [82]:
df = pd.read_csv('data2.csv') # id, user_id, username, tweet

In [83]:
df = column_dropper(df)

In [84]:
def column_dropper(df):
    x = list(df.columns)
    x.remove('id')
    x.remove('user_id')
    x.remove('username')
    x.remove('tweet')
    df.drop(columns=x, inplace= True)
    return df

In [85]:
df = column_dropper(df)

### NLP on tweets

1) remove punctuation  
2) split words  
3) remove emojis  
4) lowercase everything  
5) remove stop words  
6) tokenize words

In [86]:
from nltk.stem import LancasterStemmer, SnowballStemmer, RegexpStemmer, WordNetLemmatizer 
#this was part of the NLP notebook

import nltk
nltk.download('punkt')

#import sentence tokenizer
from nltk import sent_tokenize

#import word tokenizer
from nltk import word_tokenize

#list of stopwords
from nltk.corpus import stopwords

import string

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nabilabbas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Emoji Helper Functions

In [87]:
import emoji


def char_is_emoji(character):
    return character in emoji.UNICODE_EMOJI


def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

# NLP - Emojis (Check for and remove emojis)

In [75]:
# Checking if emoji in tokens column
df['emoji'] = df['tweet'].apply(lambda x: text_has_emoji(x))

In [76]:
df['emoji'].value_counts()

False    42389
True      3223
Name: emoji, dtype: int64

In [77]:
df['tweet'] = df['tweet'].progress_apply(lambda x: deEmojify(x))

100%|██████████| 45612/45612 [00:00<00:00, 653840.95it/s]


In [27]:
df['emoji'] = df['tweet'].apply(lambda x: text_has_emoji(x))

In [28]:
df['emoji'].value_counts()


False    45612
Name: emoji, dtype: int64

# Removing Punctuation

In [89]:
#Remove punctuations

punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")

def remove_punctuations(text):

    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

# NLP

In [90]:
def nlp(df):
    # lowercase everything
    # get rid of '\n'
    # regex remove hyperlinks
    # removing '&gt;'
    # check for emojis
    # remove emojis
    # remove punctuation
    
    # lowercase everything
    df['token'] = df['tweet'].apply(lambda x: x.lower())
    # get rid of '\n'
    df['token'] = df['token'].apply(lambda x: x.replace('\n', ' '))
    # regex remove hyperlinks
    df['token'] = df['token'].str.replace('http\S+|www.\S+', '', case=False)
    # removing '&gt;'
    df['token'] = df['token'].apply(lambda x: x.replace('&gt;', ''))
    # Checking if emoji in tokens column
    df['emoji'] = df['token'].apply(lambda x: text_has_emoji(x))
    # Removing Emojis from tokens
    df['token'] = df['token'].apply(lambda x: deEmojify(x))
    # remove punctuations
    df['token'] = df['token'].apply(remove_punctuations)
    
    return df

In [91]:
df = nlp(df)

In [93]:
# df

In [94]:
df.drop(columns = 'emoji', inplace = True)

In [96]:
# df

In [97]:
len(df)

5631

In [110]:
#lemmatize
#remove pronouns
#remove stopwords

In [111]:
df.columns

Index(['id', 'user_id', 'username', 'tweet', 'token'], dtype='object')

### Remove ' s '

In [112]:
df['token'] = df['token'].progress_apply(lambda x: str(x).replace(" s ", " "))


100%|██████████| 5631/5631 [00:00<00:00, 491011.12it/s]


In [121]:
df['token'][4330]

'spot on  davidlammy  and  nickreeves9876 keep sharing knowledge to empower people for  peoplesvote thank you'

### Lemmatize

In [108]:
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups

sp = spacy.load('en')
lookups = Lookups()
lemm = Lemmatizer(lookups)

In [109]:
sentence6 = "cult s something sticking definitely bothering narrative liberal 2018 basically moderates winning course decision sexy storyline think votes count went"

def lemma_function(text):
    dummy = []
    #this is just a test to see if it works
    for word in sp(text):
        dummy.append(word.lemma_)

    return ' '.join(dummy)
lemma_function(sentence6)

'cult s something stick definitely bother narrative liberal 2018 basically moderate win course decision sexy storyline think vote count go'

In [122]:
df['lemmatized'] = df['token'].progress_apply(lambda x: lemma_function(x))

100%|██████████| 5631/5631 [00:44<00:00, 126.74it/s]


In [131]:
df['lemmatized'][1502]

'really   all this platform be base on fear   hate   racism and protectionism     and instead of solution to climate change   that apparently do not exist for the ppc    -PRON- would promote eve more pollution and o g    -PRON- all about negativity'

### Remove Pronouns

In [132]:

df['lemmatized'] = df['lemmatized'].progress_apply(lambda x: x.replace('-PRON-', ' '))


100%|██████████| 5631/5631 [00:00<00:00, 547933.51it/s]


### Tokenize

In [133]:
df['final_text'] = df['lemmatized'].progress_apply(word_tokenize) # denier


100%|██████████| 5631/5631 [00:00<00:00, 7450.11it/s]


### Remove Stop Words

In [134]:
from nltk.corpus import stopwords
my_stopwords = set(stopwords.words('english'))

df['final_text'] = df['final_text'].progress_apply(lambda text_list: [x for x in text_list if x not in stopwords.words('english')])

100%|██████████| 5631/5631 [00:15<00:00, 368.25it/s]


### Remove Numbers from Tokenize data

In [135]:
df['final_text'] = df['final_text'].progress_apply(lambda list_data: [x for x in list_data if x.isalpha()])

100%|██████████| 5631/5631 [00:00<00:00, 282682.54it/s]


In [140]:
# df['final_text'][4330]

In [ ]:
# remove stop words
# lemmatize


#lemmatize
#remove pronouns
#remove stopwords

In [145]:
df.to_csv('data2_tokenized.csv')

In [ ]:
# upload the topic modeller
# classify what topics are associated per tweet

# vectorize
# classify tweets

# LDA - Upload the Topic Modeller

In [3]:
from wordcloud import WordCloud
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore
import matplotlib.pyplot as plt

### load pickles / DataFrame

In [4]:
file1 = open('lda3_ACTUAL.pickle','rb')
lda = pickle.load(file1)
file1.close()

file2 = open('lda_dictionary_ACTUAL.pickle','rb')
lda_dictionary = pickle.load(file2)
file2.close()

In [48]:
df = pd.read_csv('data2_tokenized.csv')

In [49]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [50]:
from ast import literal_eval
df['final_text'] = df['final_text'].progress_apply(literal_eval)

100%|██████████| 5631/5631 [00:00<00:00, 31955.08it/s]


### prepare my twitter corpus for lda

In [15]:
corpus = df['final_text']

# TOPICS HAVE TO BE DERIVED FROM MY REDDIT CORPUS

In [16]:
dictionary = Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [162]:
lda.get_document_topics(doc_term_matrix)

In [161]:
lda[doc_term_matrix]

In [164]:
df['final_text'][0]

['sign', 'petition', 'tell', 'google', 'stop', 'fund', 'climate', 'denier']

In [78]:
df['final_text'][0]

['sign', 'petition', 'tell', 'google', 'stop', 'fund', 'climate', 'denier']

In [153]:
# lda_dictionary

In [155]:
# lda.get_document_topics()

# classify what topics are associated per tweet

In [17]:
tops = []

for stuff in lda.get_document_topics(doc_term_matrix):
    tops.append(stuff)

In [22]:
tops[0][0]

(5, 0.11854284)

In [31]:
list(range(0, 15))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [32]:
df1 = pd.DataFrame(columns = list(range(0, 15)))

In [52]:
df = pd.concat([df, pd.DataFrame.from_records([{k: v for k, v in row} for row in tops])], axis = 1)


In [54]:
df.to_csv('topic_tweets.csv')

In [43]:
for doc in tops 

[[(5, 0.11854284),
  (6, 0.14605749),
  (9, 0.26941735),
  (12, 0.16221185),
  (13, 0.22967875)],
 [(6, 0.39807945), (7, 0.18191892), (9, 0.2009098), (12, 0.16265579)],
 [(2, 0.18667202), (5, 0.14760727), (6, 0.42321482), (9, 0.17580067)],
 [(1, 0.13237393), (5, 0.14396894), (6, 0.49391228), (13, 0.13806024)],
 [(2, 0.35882026), (6, 0.19930655), (9, 0.15510076), (13, 0.18188748)],
 [(2, 0.092373416), (6, 0.25881356), (7, 0.27864808), (12, 0.3212384)],
 [(2, 0.09237473), (6, 0.2587821), (7, 0.27869627), (12, 0.32122037)],
 [(2, 0.20525245), (6, 0.49588814), (11, 0.11876017), (13, 0.106724896)],
 [(2, 0.3581883), (6, 0.19989894), (9, 0.15511076), (13, 0.18191646)],
 [(1, 0.23550226), (6, 0.33928603), (12, 0.16338469), (13, 0.15704226)],
 [(2, 0.12838197), (6, 0.65837806), (13, 0.13317052)],
 [(0, 0.117197104),
  (6, 0.34389147),
  (7, 0.10242875),
  (12, 0.31724527),
  (14, 0.07478312)],
 [(0, 0.13633703), (6, 0.23360348), (12, 0.5411149)],
 [(1, 0.15842442), (6, 0.43187818), (13, 0.2954

In [39]:
for doc in tops:
    for (i,j) in doc:
        df1[i] = j
#         print(i)

In [42]:
df.drop(columns = list(range(0,15)), inplace = True)

In [29]:
for doc in tops:
    print(doc)
    
    for (i,j) in doc:
        print(i)
        print(j)
        

5
0.11854284
6
0.14605749
9
0.26941735
12
0.16221185
13
0.22967875
6
0.39807945
7
0.18191892
9
0.2009098
12
0.16265579
2
0.18667202
5
0.14760727
6
0.42321482
9
0.17580067
1
0.13237393
5
0.14396894
6
0.49391228
13
0.13806024
2
0.35882026
6
0.19930655
9
0.15510076
13
0.18188748
2
0.092373416
6
0.25881356
7
0.27864808
12
0.3212384
2
0.09237473
6
0.2587821
7
0.27869627
12
0.32122037
2
0.20525245
6
0.49588814
11
0.11876017
13
0.106724896
2
0.3581883
6
0.19989894
9
0.15511076
13
0.18191646
1
0.23550226
6
0.33928603
12
0.16338469
13
0.15704226
2
0.12838197
6
0.65837806
13
0.13317052
0
0.117197104
6
0.34389147
7
0.10242875
12
0.31724527
14
0.07478312
0
0.13633703
6
0.23360348
12
0.5411149
1
0.15842442
6
0.43187818
13
0.29540235
0
0.266566
1
0.01667166
2
0.01667171
3
0.016671646
4
0.016671648
5
0.016671646
6
0.01667176
7
0.016671702
8
0.016671654
9
0.016671686
10
0.016671646
11
0.26668355
12
0.016671658
13
0.2666904
14
0.016671646
4
0.28108406
6
0.28945026
9
0.14536603
13
0.2316779
6
0.38206166

0.40567544
4
0.22949694
6
0.18432318
8
0.119372204
0
0.23334096
2
0.15537383
6
0.34857154
9
0.102275975
14
0.118651204
0
0.2342598
3
0.09214814
6
0.21200745
7
0.2575727
9
0.08566607
13
0.07746958
0
0.49455163
3
0.09696126
6
0.106968015
12
0.1359808
14
0.10488686
0
0.14946282
1
0.21169312
6
0.13555318
8
0.2790407
9
0.1824484
0
0.35549983
1
0.075089715
2
0.13807799
4
0.051316645
6
0.053709142
8
0.20882794
10
0.094258144
0
0.14222895
1
0.396562
4
0.07734932
8
0.13805169
9
0.16181009
12
0.059967395
0
0.20072246
11
0.27515465
12
0.42383486
1
0.17140405
8
0.38646194
11
0.14206377
13
0.16104119
14
0.0876896
0
0.17039768
1
0.31477863
2
0.18304482
10
0.04315208
11
0.10364674
12
0.16071291
0
0.30460554
1
0.55092555
2
0.011113033
3
0.01111298
4
0.011112974
5
0.011112974
6
0.011113045
7
0.011112998
8
0.011112974
9
0.0111129815
10
0.011112974
11
0.011113056
12
0.01111301
13
0.011112978
14
0.011112974
0
0.15326771
1
0.2035827
2
0.04121575
8
0.16926883
9
0.114367515
10
0.034421545
11
0.16727677
12
0.

9
0.15281554
11
0.14430358
12
0.129906
4
0.3807726
6
0.3833393
8
0.13584891
6
0.4728402
8
0.4308144
6
0.47228518
8
0.41927117
1
0.093089305
2
0.23894948
8
0.2568247
9
0.192557
11
0.071677946
12
0.12177089
0
0.12840548
2
0.23582402
5
0.07633167
8
0.28291544
9
0.04669255
11
0.13725773
12
0.07108124
0
0.11242177
2
0.15517443
3
0.052611075
5
0.09146337
8
0.08404689
9
0.15325335
10
0.10575514
11
0.2236658
0
0.12478279
2
0.05458265
3
0.068399586
5
0.04738499
6
0.12783374
7
0.061281808
9
0.18319951
11
0.17148454
12
0.14271139
0
0.124773696
2
0.054582566
3
0.068397254
5
0.047384974
6
0.12783863
7
0.061279837
9
0.18319799
11
0.17148294
12
0.14272311
0
0.12619251
2
0.05702842
4
0.06965466
5
0.056420732
6
0.13261667
7
0.062280603
9
0.1791935
11
0.14481789
12
0.15347134
1
0.05017748
2
0.14989708
3
0.053528696
4
0.2557888
7
0.05316612
8
0.06322355
9
0.24503405
11
0.10877212
0
0.06375932
2
0.076928005
3
0.06316928
5
0.045236554
6
0.12289649
7
0.06133933
8
0.064804226
9
0.18996471
11
0.15768202
12
0.

8
0.06391902
10
0.16953567
11
0.086957365
13
0.045121796
0
0.09486302
1
0.04762192
2
0.18127413
3
0.046342447
4
0.08634698
6
0.14620975
8
0.15536833
10
0.12816003
11
0.053142495
13
0.04672349
0
0.22143191
1
0.121507086
6
0.1216005
8
0.19326234
9
0.11028117
10
0.040004823
11
0.12908874
14
0.04547121
1
0.17822117
6
0.42168832
8
0.3579431
0
0.05406991
1
0.20656803
3
0.046716187
5
0.063592024
6
0.166526
7
0.058865733
8
0.22541602
10
0.04439374
13
0.05904142
14
0.06086862
0
0.062369376
1
0.25887734
3
0.074051045
6
0.115296885
8
0.17192928
9
0.116727725
12
0.1784556
1
0.25462472
3
0.05766474
5
0.054444224
6
0.2555076
7
0.106602125
8
0.098685436
9
0.09763048
14
0.056018814
0
0.050471954
1
0.047850046
4
0.03808264
6
0.18799964
8
0.1374006
11
0.20220989
12
0.23741978
14
0.08345533
0
0.067547135
1
0.06646659
2
0.14921422
4
0.064678475
6
0.2609157
8
0.16509658
11
0.1946057
0
0.16007929
1
0.12631145
4
0.04907606
6
0.16173257
9
0.06794675
10
0.065607704
11
0.2053846
13
0.0446499
14
0.103153385
1
0.

0.15271966
12
0.18239264
13
0.08601451
0
0.16090831
1
0.07743018
2
0.109535255
6
0.12041457
7
0.057285327
8
0.27993998
11
0.17377673
2
0.49522775
6
0.22984503
8
0.23678859
1
0.34011152
4
0.13617052
5
0.13616978
7
0.18564461
9
0.11676699
2
0.06694071
4
0.08798774
5
0.055943146
6
0.20274311
8
0.38279745
11
0.17624408
1
0.09091363
3
0.08892725
6
0.2175662
8
0.31382152
11
0.23317231
1
0.06574144
2
0.12400736
6
0.25727773
8
0.124550745
10
0.05164331
12
0.15541168
13
0.13069998
14
0.06835614
1
0.052775323
2
0.07833431
3
0.055186015
4
0.11323956
6
0.08019704
8
0.39451134
12
0.06534266
13
0.052064285
14
0.09163662
0
0.07019679
2
0.18472917
7
0.31432742
8
0.2042144
9
0.089194216
11
0.10727267
1
0.18564096
2
0.19825687
4
0.06251533
6
0.093625315
7
0.10971149
8
0.10746549
12
0.1206014
13
0.047916573
14
0.05681711
0
0.080170184
1
0.04562093
2
0.21647629
6
0.1488976
8
0.36476204
11
0.07707455
14
0.04556789
4
0.14563738
5
0.15327084
6
0.25021031
8
0.16565612
11
0.15185814
13
0.04989501
14
0.05676808

...

13
0.14087789
1
0.24513157
7
0.1615907
10
0.14576405
11
0.35581875
0
0.15968207
6
0.5146013
7
0.12388201
9
0.12031913
2
0.088698335
6
0.2036344
7
0.10037227
8
0.38620037
10
0.11624963
11
0.071481
0
0.16025886
2
0.14223029
6
0.3825368
7
0.12373724
9
0.116927624
2
0.15187214
6
0.17688197
7
0.16122438
9
0.2655214
11
0.17773764
0
0.011114661
1
0.011114616
2
0.011114634
3
0.011114612
4
0.17768094
5
0.011114612
6
0.31577036
7
0.19637801
8
0.011114639
9
0.011114612
10
0.1879098
11
0.011114646
12
0.011114612
13
0.011114619
14
0.011114612
7
0.43992296
11
0.35391405
13
0.10613594
6
0.50804144
7
0.13359715
8
0.27829614
2
0.17499228
6
0.24639408
7
0.38780317
12
0.1241043
7
0.16869234
8
0.32887992
11
0.30173966
14
0.10899454
1
0.2268546
4
0.098129965
6
0.49503952
7
0.11329673
1
0.1810261
7
0.37895054
8
0.3511058
4
0.1799646
6
0.22963147
8
0.4017816
10
0.071124844
14
0.0729896
0
0.06389468
1
0.09796957
6
0.35008672
9
0.18141332
10
0.05939148
13
0.11407604
14
0.10344123
0
0.06171764
2
0.14332029
5
0.

0.05907917
11
0.14710158
12
0.18076754
14
0.10524786
2
0.061450753
6
0.18887065
7
0.085615166
9
0.06643962
10
0.06165697
11
0.13837501
12
0.25881025
14
0.11179933
1
0.072450325
2
0.23904517
7
0.2365983
8
0.16756031
10
0.07753959
11
0.1667746
0
0.13532019
2
0.05960694
6
0.20205136
7
0.117184184
8
0.16824473
9
0.1114298
10
0.053462327
11
0.12930322
0
0.14506266
2
0.06809518
4
0.05599603
8
0.19016549
9
0.13890465
11
0.15206547
12
0.059943274
13
0.09004583
14
0.07858041
6
0.20454407
8
0.0857654
10
0.20458256
11
0.23695724
13
0.09219374
14
0.13589622
1
0.11099427
2
0.16163333
3
0.09711356
8
0.42391655
12
0.15071578
6
0.3884986
8
0.17667289
10
0.092560515
13
0.28109136
4
0.16596013
6
0.15279289
8
0.374093
11
0.08537903
12
0.17046607
3
0.18254735
7
0.5081232
8
0.24776128
2
0.16482443
4
0.3176227
6
0.30379128
9
0.1573253
2
0.14160721
6
0.5174746
8
0.29644
2
0.24445836
6
0.31429526
8
0.12620416
10
0.06707931
12
0.20603447
3
0.08980823
5
0.07626133
6
0.33962193
9
0.20184138
11
0.08722819
12
0.16

0.11737239
9
0.32772312
11
0.19475849
12
0.12639974
4
0.14485063
6
0.2556754
8
0.24586743
13
0.13779865
14
0.13207774
2
0.32670826
6
0.14605634
8
0.2828354
11
0.16281405
0
0.013442959
1
0.22924224
2
0.013442999
3
0.013442959
4
0.013442964
5
0.013442959
6
0.013442981
7
0.013442964
8
0.013442959
9
0.20768249
10
0.013442959
11
0.40175965
12
0.013442959
13
0.013442989
14
0.013442959
0
0.0133347865
1
0.0133347865
2
0.013334801
3
0.013334783
4
0.013334815
5
0.013334783
6
0.24224216
7
0.5844055
8
0.013334793
9
0.013334836
10
0.013334783
11
0.013334811
12
0.01333479
13
0.013334784
14
0.013334783
7
0.8665735
7
0.91512096
0
0.011114069
1
0.011114053
2
0.011114063
3
0.011114061
4
0.01111405
5
0.01111405
6
0.011114094
7
0.011114066
8
0.18389036
9
0.011114061
10
0.17776565
11
0.34010488
12
0.17598447
13
0.01111405
14
0.01111405
4
0.11647342
6
0.17673595
9
0.46864042
11
0.16476436
1
0.1451562
6
0.64675164
8
0.12805948
0
0.013344322
1
0.013344311
2
0.45890424
3
0.013344307
4
0.013344307
5
0.013344307

0.10083903
6
0.505116
9
0.31336555
11
0.10873416
4
0.15241535
6
0.23565064
8
0.20083341
10
0.1522909
13
0.16349696
2
0.22895484
6
0.52238387
8
0.1596772
1
0.39866403
2
0.20155576
6
0.19117561
7
0.12692988
1
0.16195083
2
0.34697485
5
0.1335994
9
0.26558396
5
0.11851656
6
0.40995613
9
0.13717893
11
0.14163908
14
0.118585385
0
0.43960345
6
0.28184995
11
0.17839286
1
0.07933178
3
0.099181585
4
0.06659089
5
0.047925685
6
0.29757527
8
0.07757712
10
0.04887306
11
0.17836809
12
0.08635383
1
0.09372817
2
0.0562632
6
0.1920567
7
0.14950635
8
0.15975475
10
0.077330895
11
0.11878666
13
0.13307892
0
0.055158637
2
0.045447662
3
0.059561785
4
0.044579748
5
0.08284651
6
0.123290576
8
0.037397042
9
0.19567358
11
0.1760005
12
0.05550458
13
0.041412152
14
0.07508869
6
0.24825443
7
0.18740034
8
0.18146664
12
0.27036226
13
0.08644354
1
0.2815407
6
0.14544496
7
0.1350404
8
0.11537827
12
0.28746223
0
0.12102902
3
0.20098735
5
0.066520095
8
0.3372223
11
0.23246734
0
0.050695654
4
0.07805918
6
0.051216826
7
0.

11
0.16356444
12
0.12633893
0
0.42510566
4
0.097264856
6
0.11724134
9
0.1117234
10
0.09704402
14
0.0970289
6
0.49124804
8
0.24828881
9
0.09654097
10
0.09713573
0
0.26381963
2
0.1801854
8
0.18564497
10
0.106870554
14
0.19668032
3
0.115116
4
0.09772587
6
0.24699903
8
0.20293523
10
0.09701477
11
0.18563367
0
0.34024462
2
0.42885128
10
0.1419326
1
0.098704085
5
0.11709672
6
0.22469217
8
0.36639345
10
0.118836135
2
0.15621732
4
0.10659242
8
0.29007012
10
0.106704675
11
0.15600257
13
0.12438699
1
0.065949395
2
0.1647091
3
0.15893987
8
0.22386853
10
0.07631217
12
0.26729074
6
0.28790504
8
0.54231566
10
0.097013205
6
0.38441288
8
0.22289455
9
0.114006296
10
0.09705303
11
0.12097001
1
0.09020412
4
0.08891178
6
0.22615573
8
0.15881553
10
0.25613272
11
0.12965594
2
0.33962855
8
0.13762729
9
0.11346289
10
0.10888247
12
0.23357059
3
0.1602747
6
0.3133658
8
0.1362751
10
0.13341141
11
0.17328605
3
0.1397432
4
0.10809686
8
0.46180123
10
0.10690068
11
0.11664006
0
0.4202743
6
0.17423181
8
0.20530418
10

In [ ]:
# make the columns first
# function to enumerate over each document's tuples

In [86]:
df

,id,user_id,username,tweet,token,lemmatized,final_text,0,1,2,...,5,6,7,8,9,10,11,12,13,14
0,1186789645352349697,3431948722,fallennaangel,Sign the Petition - Tell Google To Stop Fundin...,sign the petition tell google to stop fundin...,sign the petition tell google to stop fund ...,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,...,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN
1,1186789063900221440,3431948722,fallennaangel,Restructuring Entire Economy Based on CO2 Clim...,restructuring entire economy based on co2 clim...,restructure entire economy base on co2 climate...,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,...,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN
2,1186722290412785664,3431948722,fallennaangel,University Dumps Professor Who Found Polar Bea...,university dumps professor who found polar bea...,university dump professor who find polar bear ...,"[university, dump, professor, find, polar, bea...",NaN,NaN,0.186672,...,0.147607,0.423215,NaN,NaN,0.175801,NaN,NaN,NaN,NaN,NaN
3,1185961657127882752,3431948722,fallennaangel,"In this climate crisis, listen to Indigenous w...",in this climate crisis listen to indigenous w...,in this climate crisis listen to indigenous ...,"[climate, crisis, listen, indigenous, wisdom, ...",NaN,0.132374,NaN,...,0.143969,0.493912,NaN,NaN,NaN,NaN,NaN,NaN,0.138060,NaN
4,1185929735194456070,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358820,...,NaN,0.199307,NaN,NaN,0.155101,NaN,NaN,NaN,0.181887,NaN
5,1185926742411743232,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092373,...,NaN,0.258814,0.278648,NaN,NaN,NaN,NaN,0.321238,NaN,NaN
6,1185926702662344704,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092375,...,NaN,0.258782,0.278696,NaN,NaN,NaN,NaN,0.321220,NaN,NaN
7,1185624556863721473,3431948722,fallennaangel,IMF Says Carbon Tax Is Most Powerful Way to Fi...,imf says carbon tax is most powerful way to fi...,imf say carbon tax be most powerful way to fig...,"[imf, say, carbon, tax, powerful, way, fight, ...",NaN,NaN,0.205252,...,NaN,0.495888,NaN,NaN,NaN,NaN,0.118760,NaN,0.106725,NaN
8,1184227872376012801,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358188,...,NaN,0.199899,NaN,NaN,0.155111,NaN,NaN,NaN,0.181916,NaN
9,1183940137618673664,3431948722,fallennaangel,500 Scientists Sign Letter to UN: ‘There Is No...,500 scientists sign letter to un there is no ...,500 scientist sign letter to un there be no ...,"[scientist, sign, letter, un, climate, emergency]",NaN,0.235502,NaN,...,NaN,0.339286,NaN,NaN,NaN,NaN,NaN,0.163385,0.157042,NaN


# START CLASSIFYING

### upload the classifier

In [55]:
file = open('LR_monday2.pickle','rb')
lr = pickle.load(file)
file.close()

### VECTORIZE TOKENS

In [88]:
#count vectorizer
file3 = open('count_vectorizer_final.pickle','rb')
cv = pickle.load(file3)
file3.close()

#column names
file4 = open('column_names_final.pickle','rb')
cv_columns = pickle.load(file4)
file4.close()

In [64]:
X_all = df[['final_text']]

In [90]:
wm = cv.transform([' '.join(x) for x in tqdm_notebook(X_all.final_text.tolist())])

In [ ]:
[' '.join(x) for x in tqdm_notebook(X_all.final_text.tolist())]

In [75]:
cv = CountVectorizer(min_df = 2, ngram_range=(1,2))
wm = cv.fit_transform([' '.join(x) for x in tqdm_notebook(X_all.final_text.tolist())])
columns = cv.get_feature_names()

In [87]:
# lr.predict(['you', 'greta'])

In [81]:
# for x in X_all.final_text.tolist():
#     print([' '.join(x)])
#     print('')

['sign petition tell google stop fund climate denier']

['restructure entire economy base climate hoax top scientist explain w charles shults']

['university dump professor find polar bear thrive despite climate change']

['climate crisis listen indigenous wisdom broadview magazine']

['expert destroy climate change hoax minute']

['climate change hoax collapse new science find human activity virtually zero impact global temperature']

['climate change hoax collapse new science find human activity virtually zero impact global temperature']

['imf say carbon tax powerful way fight climate crisis']

['expert destroy climate change hoax minute']

['scientist sign letter un climate emergency']

['extinction rebellion shut time square climate activist superglue boat']

['fossil fuel firm spend million social medium ad climate regulation portray self green hero']

['sander say population control would part climate agenda']

['meet money behind climate denial movement']

['year climate scare'


['solution mitigate climateemergency would scheer government contribute andy even believe climate change please give andyamerican power make thing bad cdnpoli']

['con bot troll may create illusion well chance ndp reality liberal party stopscheer trudeau embody canadian value work climate healthcare equity true con destroy love cdnpoli']

['point history afford climate change deny scheer government power must work together find solution scheer obviously interested mitigate crisis even admit problem bigoil cdnpoli']

['opinion exxon deceive investor climate change']

['disarm case act right climate change greta thunberg via youtube']

['via nytimes forest elephant defend climate change']

['fact carbonmonixide co toxic highly inflamable small dosis lead death climate killer however carbondioxide inert gas harm except use extinguish fire displace oxygen']

['ask would go new york september united nations climate summit invite greta say would fly could sail ship learn go container ship w

['fourth generation nuclear energy must take seriously fight climate change']

['understand urgency climate change many move position deep distrust nuclear power acknowledgement combination wind solar ccs nuclear must play serious role climate solution']

['ieabirol presentation morning clear wake call tackle climate change use lowcarbon altogether time skirmish nuclear renewable foratom nuclear']

['nuclear energy key combat climate change']

['foratom highlight need investment low carbon technology meet climate challenge']

['dirty secret renewable advocate protect fossil fuel interest climate via forbes']

['summit leak reveal eu rift climate change euractiv']

['drama become pro nuclear name climate pic twitter com']

['finland want eu agree plan net zero carbon footprint must align climate energy policy accord deg target idea back country like sweden france netherlands oppose like germany poland']

['attend behalf policy team nttmenergycity great back begin uongeography hear one o


['stop waste time senial look lecture primary role climate change earth history listen scientist fossil fuel industry']

['lead within next century dangerous weather climate effect current biosphere develop condition civilisation']

['definately sound like climate change denier answer']

['michael emission big threat earth ecosystem civilisation either climate change deni person understand threat climate change posse solar energy wind far clean coal oil']

['wow reason shelley climate change denial nonsense also endanger also wrong one stop deny reality']

['oh cut nonsense primary forcer climate change throughtout earth geological history please stop waste time ignorance watch lecture role']

['mit climate science department support science behind agw post date either old position hold mit simply nonsense mit late position']

['also claim climate change past natural blame current change false logic think pump billion atmosphere every year increse effect']

['stop waste time denial wa


['doomsday happen think tank compile decade worth dire climate prediction politician talk climate change see statement freeman dyson see real science say climate see junk science']

['even bad climate change candidate know nothing tell people conceal fact model wrong competent scientist see uncorrrupted datum want base policy junk science']

['amazed people know absolutely nothing climate change make statement even aware anthropogenic model prove wrong call science junk science one really understand cause effect']

['warren understand climate change look datum way anthropogenic proponent distort datum make believe model base credible science hear phrase science prove almost make vomit']

['pity warren people understand nothing stop climate change earth influence small compare mother nature focus pollution preserve rain forest']

['understand climate change limited role play people hear proponent anthropogenic model know model prove wrong science junk science propose policy base fraud'

In [93]:
tweet_classes = lr.predict(wm)

### RUN THROUGH THE CLASSIFIER

In [ ]:
# dictionary to be count vectorized needs to be my other reddit

In [ ]:
cv.transform

In [95]:
df['class'] = pd.Series(tweet_classes)

In [98]:
# df['class'].value_counts()

In [100]:
df.to_csv('classified_tweets_preagg.csv')

In [101]:
df

,id,user_id,username,tweet,token,lemmatized,final_text,0,1,2,...,6,7,8,9,10,11,12,13,14,class
0,1186789645352349697,3431948722,fallennaangel,Sign the Petition - Tell Google To Stop Fundin...,sign the petition tell google to stop fundin...,sign the petition tell google to stop fund ...,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,...,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1
1,1186789063900221440,3431948722,fallennaangel,Restructuring Entire Economy Based on CO2 Clim...,restructuring entire economy based on co2 clim...,restructure entire economy base on co2 climate...,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,...,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1
2,1186722290412785664,3431948722,fallennaangel,University Dumps Professor Who Found Polar Bea...,university dumps professor who found polar bea...,university dump professor who find polar bear ...,"[university, dump, professor, find, polar, bea...",NaN,NaN,0.186672,...,0.423215,NaN,NaN,0.175801,NaN,NaN,NaN,NaN,NaN,1
3,1185961657127882752,3431948722,fallennaangel,"In this climate crisis, listen to Indigenous w...",in this climate crisis listen to indigenous w...,in this climate crisis listen to indigenous ...,"[climate, crisis, listen, indigenous, wisdom, ...",NaN,0.132374,NaN,...,0.493912,NaN,NaN,NaN,NaN,NaN,NaN,0.138060,NaN,0
4,1185929735194456070,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358820,...,0.199307,NaN,NaN,0.155101,NaN,NaN,NaN,0.181887,NaN,0
5,1185926742411743232,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092373,...,0.258814,0.278648,NaN,NaN,NaN,NaN,0.321238,NaN,NaN,0
6,1185926702662344704,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092375,...,0.258782,0.278696,NaN,NaN,NaN,NaN,0.321220,NaN,NaN,0
7,1185624556863721473,3431948722,fallennaangel,IMF Says Carbon Tax Is Most Powerful Way to Fi...,imf says carbon tax is most powerful way to fi...,imf say carbon tax be most powerful way to fig...,"[imf, say, carbon, tax, powerful, way, fight, ...",NaN,NaN,0.205252,...,0.495888,NaN,NaN,NaN,NaN,0.118760,NaN,0.106725,NaN,0
8,1184227872376012801,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358188,...,0.199899,NaN,NaN,0.155111,NaN,NaN,NaN,0.181916,NaN,0
9,1183940137618673664,3431948722,fallennaangel,500 Scientists Sign Letter to UN: ‘There Is No...,500 scientists sign letter to un there is no ...,500 scientist sign letter to un there be no ...,"[scientist, sign, letter, un, climate, emergency]",NaN,0.235502,NaN,...,0.339286,NaN,NaN,NaN,NaN,NaN,0.163385,0.157042,NaN,0


In [ ]:
# Aggregate to a per topic per tweet repeats

In [103]:
pd.set_option('display.max_columns', None)

In [105]:
df[df['user_id']== 3431948722]

,id,user_id,username,tweet,token,lemmatized,final_text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,class
0,1186789645352349697,3431948722,fallennaangel,Sign the Petition - Tell Google To Stop Fundin...,sign the petition tell google to stop fundin...,sign the petition tell google to stop fund ...,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1
1,1186789063900221440,3431948722,fallennaangel,Restructuring Entire Economy Based on CO2 Clim...,restructuring entire economy based on co2 clim...,restructure entire economy base on co2 climate...,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1
2,1186722290412785664,3431948722,fallennaangel,University Dumps Professor Who Found Polar Bea...,university dumps professor who found polar bea...,university dump professor who find polar bear ...,"[university, dump, professor, find, polar, bea...",NaN,NaN,0.186672,NaN,NaN,0.147607,0.423215,NaN,NaN,0.175801,NaN,NaN,NaN,NaN,NaN,1
3,1185961657127882752,3431948722,fallennaangel,"In this climate crisis, listen to Indigenous w...",in this climate crisis listen to indigenous w...,in this climate crisis listen to indigenous ...,"[climate, crisis, listen, indigenous, wisdom, ...",NaN,0.132374,NaN,NaN,NaN,0.143969,0.493912,NaN,NaN,NaN,NaN,NaN,NaN,0.138060,NaN,0
4,1185929735194456070,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358820,NaN,NaN,NaN,0.199307,NaN,NaN,0.155101,NaN,NaN,NaN,0.181887,NaN,0
5,1185926742411743232,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092373,NaN,NaN,NaN,0.258814,0.278648,NaN,NaN,NaN,NaN,0.321238,NaN,NaN,0
6,1185926702662344704,3431948722,fallennaangel,Climate change hoax COLLAPSES as new science f...,climate change hoax collapses as new science f...,climate change hoax collapse as new science fi...,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092375,NaN,NaN,NaN,0.258782,0.278696,NaN,NaN,NaN,NaN,0.321220,NaN,NaN,0
7,1185624556863721473,3431948722,fallennaangel,IMF Says Carbon Tax Is Most Powerful Way to Fi...,imf says carbon tax is most powerful way to fi...,imf say carbon tax be most powerful way to fig...,"[imf, say, carbon, tax, powerful, way, fight, ...",NaN,NaN,0.205252,NaN,NaN,NaN,0.495888,NaN,NaN,NaN,NaN,0.118760,NaN,0.106725,NaN,0
8,1184227872376012801,3431948722,fallennaangel,Expert Destroys Climate Change Hoax in 3 Minut...,expert destroys climate change hoax in 3 minut...,expert destroy climate change hoax in 3 minute,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358188,NaN,NaN,NaN,0.199899,NaN,NaN,0.155111,NaN,NaN,NaN,0.181916,NaN,0
9,1183940137618673664,3431948722,fallennaangel,500 Scientists Sign Letter to UN: ‘There Is No...,500 scientists sign letter to un there is no ...,500 scientist sign letter to un there be no ...,"[scientist, sign, letter, un, climate, emergency]",NaN,0.235502,NaN,NaN,NaN,NaN,0.339286,NaN,NaN,NaN,NaN,NaN,0.163385,0.157042,NaN,0


In [106]:
df.drop(columns = ['token', 'lemmatized', 'tweet'], inplace = True)

In [107]:
df

,id,user_id,username,final_text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,class
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1
1,1186789063900221440,3431948722,fallennaangel,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1
2,1186722290412785664,3431948722,fallennaangel,"[university, dump, professor, find, polar, bea...",NaN,NaN,0.186672,NaN,NaN,0.147607,0.423215,NaN,NaN,0.175801,NaN,NaN,NaN,NaN,NaN,1
3,1185961657127882752,3431948722,fallennaangel,"[climate, crisis, listen, indigenous, wisdom, ...",NaN,0.132374,NaN,NaN,NaN,0.143969,0.493912,NaN,NaN,NaN,NaN,NaN,NaN,0.138060,NaN,0
4,1185929735194456070,3431948722,fallennaangel,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358820,NaN,NaN,NaN,0.199307,NaN,NaN,0.155101,NaN,NaN,NaN,0.181887,NaN,0
5,1185926742411743232,3431948722,fallennaangel,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092373,NaN,NaN,NaN,0.258814,0.278648,NaN,NaN,NaN,NaN,0.321238,NaN,NaN,0
6,1185926702662344704,3431948722,fallennaangel,"[climate, change, hoax, collapse, new, science...",NaN,NaN,0.092375,NaN,NaN,NaN,0.258782,0.278696,NaN,NaN,NaN,NaN,0.321220,NaN,NaN,0
7,1185624556863721473,3431948722,fallennaangel,"[imf, say, carbon, tax, powerful, way, fight, ...",NaN,NaN,0.205252,NaN,NaN,NaN,0.495888,NaN,NaN,NaN,NaN,0.118760,NaN,0.106725,NaN,0
8,1184227872376012801,3431948722,fallennaangel,"[expert, destroy, climate, change, hoax, minute]",NaN,NaN,0.358188,NaN,NaN,NaN,0.199899,NaN,NaN,0.155111,NaN,NaN,NaN,0.181916,NaN,0
9,1183940137618673664,3431948722,fallennaangel,"[scientist, sign, letter, un, climate, emergency]",NaN,0.235502,NaN,NaN,NaN,NaN,0.339286,NaN,NaN,NaN,NaN,NaN,0.163385,0.157042,NaN,0


In [108]:
df.isnull().sum()

id               0
user_id          0
username         0
final_text       0
0             3246
1             3549
2             3404
3             4029
4             3414
5             4326
6             1182
7             3818
8             2064
9             3089
10            3445
11            2944
12            3290
13            4004
14            3416
class            0
dtype: int64

In [112]:
df[4].count()

2217

In [117]:
# df1 = df.drop(columns = ['id', 'user_id', 'username', 'final_text', 'class'])

In [118]:
len(list(df.columns))

20

In [120]:
# 20 columns
# 15 are my topics

# 10 columns have values
# 10 columns are null
# nulls only exist in topics

#therefore 15 - x.count() = number of repeats

#how many topics do i have 

In [124]:
#make this column
df.apply(lambda x: x.count(), axis=1).value_counts()

#repeat the tweet that many time

#Group by User and topic

#df['topic']

#df['probability']

 4    1091
 5    1076
 6     918
 3     762
 2     517
 7     504
-5     387
 1     209
 8      83
 0      64
-1      13
 9       5
-2       1
-3       1
dtype: int64

In [128]:
df['topic_counts'] = df.drop(columns = ['id', 'user_id', 'username', 'final_text', 'class']).apply(lambda x: x.count(), axis=1)#.value_counts()


In [136]:
sum(df.drop(columns = ['id', 'user_id', 'username', 'final_text', 'class', 'topic_counts']).apply(lambda x: x.count(), axis=1))#.value_counts()


35245

In [134]:
df['to']

35245

### Repeat the rows based off of topic counts

In [137]:
df = df.loc[np.repeat(df.index.values, df.topic_counts)]


In [138]:
df

,id,user_id,username,final_text,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,class,topic_counts
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1,5
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1,5
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1,5
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1,5
0,1186789645352349697,3431948722,fallennaangel,"[sign, petition, tell, google, stop, fund, cli...",NaN,NaN,NaN,NaN,NaN,0.118543,0.146057,NaN,NaN,0.269417,NaN,NaN,0.162212,0.229679,NaN,1,5
1,1186789063900221440,3431948722,fallennaangel,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1,4
1,1186789063900221440,3431948722,fallennaangel,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1,4
1,1186789063900221440,3431948722,fallennaangel,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1,4
1,1186789063900221440,3431948722,fallennaangel,"[restructure, entire, economy, base, climate, ...",NaN,NaN,NaN,NaN,NaN,NaN,0.398079,0.181919,NaN,0.200910,NaN,NaN,0.162656,NaN,NaN,1,4
2,1186722290412785664,3431948722,fallennaangel,"[university, dump, professor, find, polar, bea...",NaN,NaN,0.186672,NaN,NaN,0.147607,0.423215,NaN,NaN,0.175801,NaN,NaN,NaN,NaN,NaN,1,4


## create the columns


In [156]:
len(list(df['id'].unique()))

5551

In [154]:
df.drop_duplicates(subset = 'id', inplace = True)

5551

In [155]:
len(tops)

5631

In [ ]:
d = {'identifier1':[('date1','value1'),('date2','value2')],
     'identifier2':[('date1','value1'),('date3','value3'),('date4','value4')]}

L = [(k, *t) for k, v in d.items() for t in v]

df = pd.DataFrame(L, columns=['identifier','date','val'])

## group by the new columns

In [145]:
df.to_csv('topic_prob_columns_gotta_add_then_groupby.csv')

In [ ]:
#dictionary
for topics in documnet
{document: [topics]}

In [ ]:
df['topic'] = 
df['topic_probability'] = 

In [143]:
for index, row in df.iterrows():
    print(row[0], index)

nan 0
nan 0
nan 0
nan 0
nan 0
nan 1
nan 1
nan 1
nan 1
nan 2
nan 2
nan 2
nan 2
nan 3
nan 3
nan 3
nan 3
nan 4
nan 4
nan 4
nan 4
nan 5
nan 5
nan 5
nan 5
nan 6
nan 6
nan 6
nan 6
nan 7
nan 7
nan 7
nan 7
nan 8
nan 8
nan 8
nan 8
nan 9
nan 9
nan 9
nan 9
nan 10
nan 10
nan 10
0.11719710379838943 11
0.11719710379838943 11
0.11719710379838943 11
0.11719710379838943 11
0.11719710379838943 11
0.1363370269536972 12
0.1363370269536972 12
0.1363370269536972 12
nan 13
nan 13
nan 13
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
0.2665660083293915 14
nan 15
nan 15
nan 15
nan 15
nan 16
nan 16
nan 16
nan 17
nan 17
nan 17
nan 17
nan 18
nan 18
nan 18
nan 18
nan 18
nan 19
nan 19
nan 19
nan 20
nan 20
nan 20
nan 20
nan 21
nan 21
nan 21
nan 21
nan 21
nan 2

nan 179
nan 179
nan 179
nan 180
nan 180
nan 180
nan 180
nan 180
nan 180
nan 181
nan 181
nan 181
nan 181
nan 181
nan 181
nan 181
nan 181
nan 182
nan 182
nan 182
nan 182
nan 183
nan 183
nan 183
nan 183
nan 183
nan 183
0.3401543200016022 184
0.3401543200016022 184
0.3401543200016022 184
nan 185
nan 185
nan 185
0.2389463484287262 186
0.2389463484287262 186
0.2389463484287262 186
0.2389463484287262 186
0.2389463484287262 186
0.2389463484287262 186
0.2389463484287262 186
nan 187
nan 187
nan 187
nan 187
nan 187
nan 187
nan 188
nan 188
nan 188
nan 188
nan 188
nan 189
nan 189
nan 189
nan 189
nan 189
0.1011703759431839 190
0.1011703759431839 190
0.1011703759431839 190
0.1011703759431839 190
0.1011703759431839 190
0.1011703759431839 190
0.1011703759431839 190
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.10357436537742615 191
0.21337680518627167 192
0.21337680518627167 192
0

nan 354
nan 355
nan 355
nan 355
nan 355
nan 356
nan 356
nan 356
nan 356
nan 356
nan 357
nan 357
nan 357
nan 357
nan 357
nan 357
nan 357
nan 358
nan 358
nan 358
nan 358
nan 358
nan 358
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
0.057838477194309235 359
nan 360
nan 360
nan 360
nan 360
nan 360
nan 360
nan 360
nan 360
nan 360
nan 360
nan 361
nan 361
nan 361
nan 361
nan 361
nan 362
nan 362
nan 362
nan 362
nan 363
nan 363
nan 363
nan 363
nan 363
0.1622982621192932 364
0.1622982621192932 364
0.1622982621192932 364
0.1622982621192932 364
0.1622982621192932 364
0.1622982621192932 364
0.17243745923042297 365
0.17243745923042297 365
0.17243745923042297 365
0.17243745923042297 365
0.17243745923042297 365
0.17243745923042297 365
nan 366
nan 366
nan 366
nan 366
nan 366
nan 366
nan 366
0.12077837437391281 367
0.12077837437391281 367
0.12077837437391281 367
0.120778374373

0.05903153121471405 511
0.05903153121471405 511
nan 512
nan 512
nan 512
nan 512
nan 512
nan 512
nan 512
nan 513
nan 513
nan 513
nan 513
nan 513
nan 513
nan 513
nan 513
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.2118510752916336 514
0.23506931960582733 515
0.23506931960582733 515
0.23506931960582733 515
0.23506931960582733 515
nan 516
nan 516
nan 516
nan 516
nan 516
nan 516
nan 516
0.24270813167095184 517
0.24270813167095184 517
0.24270813167095184 517
0.24270813167095184 517
0.24270813167095184 517
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.1366884410381317 518
0.09089762717485428 519
0.09089762717485428 519
0.09089762717485428 519
0.09089762717485428 519
0.09089762717485428 519
0.09089762717485428 519
0.

0.17837068438529968 660
0.17837068438529968 660
0.17837068438529968 660
0.17837068438529968 660
nan 661
nan 661
nan 661
nan 661
nan 661
nan 661
nan 661
0.09914696216583252 662
0.09914696216583252 662
0.09914696216583252 662
0.09914696216583252 662
0.09914696216583252 662
nan 663
nan 664
nan 664
nan 664
nan 664
nan 665
nan 665
nan 665
nan 665
0.2325003445148468 666
0.2325003445148468 666
0.2325003445148468 666
0.2325003445148468 666
0.2325003445148468 666
0.2486356645822525 667
0.2486356645822525 667
0.2486356645822525 667
0.2486356645822525 667
0.2486356645822525 667
nan 668
nan 668
nan 668
nan 669
nan 669
nan 669
nan 669
nan 669
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.013341308571398258 670
0.0133413085

...

nan 5020
nan 5020
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
0.18941882252693176 5021
nan 5022
nan 5022
nan 5022
nan 5022
nan 5022
nan 5023
nan 5023
nan 5023
nan 5023
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
0.011112548410892487 5024
nan 5025
nan 5025
nan 5025
nan 5025
nan 5025
nan 5025
nan 5025
0.14641378819942474 5026
0.14641378819942474 5026
0.14641378819942474 5026
0.14641378819942474 5026
0.14641378819942474 5026
0.14641378819942474 5026
0.011114849708974361 5027
0.011114849708974361 5027
0.011114849708974361 5027
0.011114849708974361

0.07586368918418884 5174
0.07586368918418884 5174
0.07586368918418884 5174
0.06756690889596939 5175
0.06756690889596939 5175
0.06756690889596939 5175
0.06756690889596939 5175
0.06756690889596939 5175
0.06756690889596939 5175
0.06756690889596939 5175
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
0.0842936635017395 5176
nan 5177
nan 5177
nan 5177
nan 5177
nan 5177
0.13376697897911072 5178
0.13376697897911072 5178
0.13376697897911072 5178
0.13376697897911072 5178
0.13376697897911072 5178
0.13376697897911072 5178
0.13376697897911072 5178
0.13378475606441498 5179
0.13378475606441498 5179
0.13378475606441498 5179
0.13378475606441498 5179
0.13378475606441498 5179
0.13378475606441498 5179
0.13378475606441498 5179
0.10549233108758926 5180
0.10549233108758926 5180
0.10549233108758926 5180
0.10549233108758926 5180
0.10549233108758926 5180
0.10549233108758926 5180
0.10549233108

0.24791617691516876 5316
0.24791617691516876 5316
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.2620457112789154 5317
0.1989118903875351 5318
0.1989118903875351 5318
0.1989118903875351 5318
0.1989118903875351 5318
0.1989118903875351 5318
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.5466054081916809 5319
0.17430558800697327 5320
0.17430558800697327 5320
0.17430558800697327 5320
0.17430558800697327 5320
0.27844220

0.4202742874622345 5471
0.4202742874622345 5471
0.4202742874622345 5471
nan 5472
nan 5472
nan 5472
nan 5472
nan 5473
nan 5473
nan 5473
nan 5473
nan 5473
nan 5474
nan 5474
nan 5474
nan 5474
nan 5474
nan 5474
0.12269764393568039 5475
0.12269764393568039 5475
0.12269764393568039 5475
0.12269764393568039 5475
0.12269764393568039 5475
0.12269764393568039 5475
0.09050368517637253 5476
0.09050368517637253 5476
0.09050368517637253 5476
0.09050368517637253 5476
0.09050368517637253 5476
0.09050368517637253 5476
nan 5477
nan 5477
nan 5477
nan 5477
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
0.08941998332738876 5478
nan 5479
nan 5479
nan 5479
nan 5479
nan 5480
nan 5480
nan 5480
nan 5480
nan 5480
nan 5480
0.08939742296934128 5481
0.08939742296934128 5481
0.08939742296934128 5481
0.08939742296934128 5481
0.08939742296934128 5481
0.08939742296934128 5481
0.089397422969341

0.1093609631061554 5620
0.1093609631061554 5620
0.1093609631061554 5620
0.27394944429397583 5621
0.27394944429397583 5621
0.27394944429397583 5621
0.27394944429397583 5621
0.27394944429397583 5621
0.27394944429397583 5621
0.27396360039711 5622
0.27396360039711 5622
0.27396360039711 5622
0.27396360039711 5622
0.27396360039711 5622
0.27396360039711 5622
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
0.13962075114250183 5623
nan 5624
nan 5624
nan 5624
nan 5624
nan 5624
nan 5625
nan 5625
nan 5625
nan 5625
nan 5625
nan 5625
nan 5626
nan 5626
nan 5626
nan 5626
nan 5626
nan 5626
nan 5627
nan 5627
nan 5627
nan 5627
nan 5627
nan 5627
0.10930661857128143 5628
0.10930661857128143 5628
0.10930661857128143 5628
0.10930661857128143 5628
0.10930661857128143 5628
0.10930661857128143 5628
nan 5629
nan 5629
nan 5629
nan 5629
nan 5629
nan 5629
nan 5630
nan 5630
nan 5630
nan 5630

In [ ]:
#check for nans